In [3]:
import kaggle as kg
import os
import pandas as pd
import numpy as np
import matplotlib as mp

os.environ['KAGGLE_USERNAME'] ='taylorjmurray' #os.environ is a mapping object, think dictionary. os.environ is created as soon as os module is imported.
os.environ['KAGGLE_KEY'] = '9514abe911ad574d3d1f13f9dfac480e'
kg.api.authenticate()
download_path = "/Users/tayma/datasets" # Windows 
#download_path = "/Users/taylormurray/datasets" #MaciOS
kg.api.dataset_download_files(dataset= 'austinreese/usa-housing-listings', path = download_path, unzip = True)
file_path = os.path.join(download_path, 'housing.csv') #joins the download_path and 'housing.csv' and returns it as a new path




Dataset URL: https://www.kaggle.com/datasets/austinreese/usa-housing-listings


In [4]:

db = pd.read_csv(file_path)
db.info() #To get a sense of the contents of the data- in particular its features.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384977 entries, 0 to 384976
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       384977 non-null  int64  
 1   url                      384977 non-null  object 
 2   region                   384977 non-null  object 
 3   region_url               384977 non-null  object 
 4   price                    384977 non-null  int64  
 5   type                     384977 non-null  object 
 6   sqfeet                   384977 non-null  int64  
 7   beds                     384977 non-null  int64  
 8   baths                    384977 non-null  float64
 9   cats_allowed             384977 non-null  int64  
 10  dogs_allowed             384977 non-null  int64  
 11  smoking_allowed          384977 non-null  int64  
 12  wheelchair_access        384977 non-null  int64  
 13  electric_vehicle_charge  384977 non-null  int64  
 14  come

## About the features:

* ``'id'`` - an integer acting as an id number.
* ``'url'`` - the url hosting the listing of the house. 
* ``'region'`` - location of house (usually a city name).
* ``'region_url'`` - the regions craigslist url 
* ``'price'`` - the rent/price *per month* in US dollars for the house.
* ``'type'`` - the type of the house whose values consist of
    - apartment
    - condo
    - house
    - duplex
    - townhouse
    - loft
    - manufactured
    - cottage/cabin
    - flat
    - in-law
    - land
    - assisted living
    
* ``'sqfeet'`` - the square footage of the house.
* ``'beds'`` - the number of bedrooms in the house.
* ``'baths'`` - the number of bathrooms in the house.
* ``'cats_allowed'`` - Boolean indicating whether or not cats are allowed in the house (1 = yes, 0 = no).
* ``'dogs_allowed'`` - Boolean indicating whether or not dogs are allowed in the house (1 = yes, 0 = no).
* ``'smoking_allowed'`` - Boolean indicating whether or not smoking is allowed in the house (1 = yes, 0 = no).
* ``'wheelchair_access'`` - Boolean indicating whether or not the house is wheelchair accessible  (1 = yes, 0 = no).
* ``'electric_vehicle_charge'`` - Boolean indicating whether or not the house is equipped with electric vehicle charge capabilities (1 = yes, 0 = no).
* ``'comes_furnished'`` - Boolean indicating whether or not the house is furnished (1 = yes, 0 = no).
* ``'laundry_options'`` - laundry option available in the house whose values consist of: 
    - w/d in unit
    - w/d hookups
    - laundry on site
    - laundry in bldg
    - nan
    - no laundry on site
* ``'parking_options'`` - parking option available in the house whose values consist of:
    - carport
    - attached garage
    - nan
    - off-street parking
    - detached garage
    - street parking
    - no parking
    - valet parking
* ``'image_url'`` - url that hosts the image of the house.
* ``'description'`` - a description of the house.
* ``'lat'`` - the latitude coordinate for the house.
* ``'long'`` - longitude coordinate for the house.
* ``'state'`` - the state (abbreviated) in which the house resides.

## Goal:

The goal for this project is to predict the price per month (in US dollars) of a house based on its:

* square footage ``'sqfeet'``
* number of beds ``'beds'``
* number of baths ``'baths'``
* housing type ``'type'``
* pet policies ``'cats_allowed'`` and ``'dogs_allowed'``
* smoking policies ``'smoking_allowed'``
* laundry options ``'laundry_options'``
* parking options ``'parking_options'``
* location ``'state'``

To achieve this we begin with exploring the data in more depth to gain an understanding on the relationships between the features we have just selected and our target (label) ``price``. We start by setting ``'id'`` as the index for a copy of the data base as the values of the ``'id'`` feature are unique and contain no N/A values; hence, the values in ``'id'`` serve as unique identifiers. One can check the validity of the previous statements by running the following python code

```python

db['id'].nunique() == db.index.size # Checks if the number of unique values in the 'id' column is equal to the size of the index. Return True
db['id'].isnan().value_counts() # Shows that there are no nan values in 'id' column.
```

  We next employ the simple, yet informative, ``.describe()`` method. 

In [75]:
db = db.copy() # We do not want to change the original database
# Select the features that are deemed relavent.
selected_features =['id',
                    'price',
                    'sqfeet', 
                    'beds', 'baths',
                    'type',
                    'cats_allowed',
                    'dogs_allowed',
                    'smoking_allowed',
                    'laundry_options',
                    'parking_options',
                    'state']

data = db[selected_features].reset_index(drop = True).set_index('id') # Gets rid of original index (if there is one) and sets the 'id' column to be the new index.

data.describe() # Displays statistical information 


,price,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed
count,3.849770e+05,3.849770e+05,384977.000000,384977.000000,384977.000000,384977.000000,384977.000000
mean,8.825722e+03,1.059900e+03,1.905345,1.480718,0.726890,0.707918,0.731771
std,4.462200e+06,1.915076e+04,3.494572,0.618061,0.445557,0.454721,0.443038
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.050000e+02,7.500000e+02,1.000000,1.000000,0.000000,0.000000,0.000000
50%,1.036000e+03,9.490000e+02,2.000000,1.000000,1.000000,1.000000,1.000000
75%,1.395000e+03,1.150000e+03,2.000000,2.000000,1.000000,1.000000,1.000000
max,2.768307e+09,8.388607e+06,1100.000000,75.000000,1.000000,1.000000,1.000000


Here we see some unusual properties in the some our our features columns as well as our target column:

* In the ``'price'`` column there is a minimum value of $\$0$ per month. We will assume instances with a price of $\$0$ per month represent bad data and delete such instances.
* In the ``'price'`` column there is a maximum value of $\$2,768,307,249$ per month(```python data['price'].max()``` to get this exact value). Likely, this is some sort of value error- a good indicator is that the 75th quantile $\$1,395$ per month. We will deal with high values like this via outlier detection. One possible argument is that high values like this may be the total cost of a house rather than the price per month; however, this too is unlikely as $2,768,307,249$ is in the billions.
* In the ``'sqfeet'`` column there is a minimum value of $0 ~\mathrm{ft}^2$. We will assume instances with a square footage of $ 0 ~\mathrm{ft}^2$ represent bad data and delete such instances
* In the ``'sqfeet'`` column there is a maximum value of $ 8388607 ~\mathrm{ft}^2 $. Likely this is some sort of data entry error as a quick google search shows that the house with the largest square footage has $178,926~\mathrm{ft}^2$.
* In the ``'beds'`` column there is a minimum value of $0$ beds. We will assume instances with $0$ beds represent bad data and delete such instances.
* In the ``'beds'`` column there is a maximum value of $1100$ beds. Likely this represents bad data and such instances will be dealt with via outlier detection.
* In the ``'baths'`` column there is a minimum value of $0$ baths. We will assume instances with $0$ baths represent bad data and delete such instances.
* In the ``'baths'`` column there is a maximum value of $75$ baths. Likely this represents bad data and such instance will be dealt with via outlier detection.

With the limit information displayed by ``.describe()`` the values found in the features ``'cats_allowed'``, ``'dogs_allowed'``, and ``'smoking_allowed'`` make sense while keeping in mind they are one-hot encoded; however, we will study these in more detail soon via histograms and box plots. Before doing so, we run the following code to count the number of na values for each of our selected features

```python
size_idx = data.index.size
for col in data.columns:
    na_col = data[col].isna()
    na_val = na_col.values 
    num_na = na_col.sum()
    
    print(f'{col} contains {num_na} na values. Approximately {num_na/size_idx} values in {col} are na.')
```
```output
price contains 0 na values. Approximately 0.0 values in price are na.
sqfeet contains 0 na values. Approximately 0.0 values in sqfeet are na.
beds contains 0 na values. Approximately 0.0 values in beds are na.
baths contains 0 na values. Approximately 0.0 values in baths are na.
type contains 0 na values. Approximately 0.0 values in type are na.
cats_allowed contains 0 na values. Approximately 0.0 values in cats_allowed are na.
dogs_allowed contains 0 na values. Approximately 0.0 values in dogs_allowed are na.
smoking_allowed contains 0 na values. Approximately 0.0 values in smoking_allowed are na.
laundry_options contains 79026 na values. Approximately 0.20527460082030874 values in laundry_options are na.
parking_options contains 140687 na values. Approximately 0.365442610857272 values in parking_options are na.
state contains 0 na values. Approximately 0.0 values in state are na.
```
From this we see that ``'laundry_options'`` and ``'parking_options'`` are the only features that contain na values. It is important to note here that values in the ``'laundry_options'`` and ``'parking_options'`` have 7 and 8 unique values, respectively. Moreover, both ``'laundry_options'`` and ``'parking_options'`` contain a ``no laundry on site`` and ``no parking`` value, respectively, so an na value in either of these columns **does not mean** that there is no laundry or parking options, just that it is truly unknown. 

**An assumption**: For handling na values in both ``'laundry_options'`` and ``'parking_options'``, we will replace each na value by ``no laundry on site`` and ``no parking``, respectively. Our reasoning here is that it is a possibility that 'na' was taken to mean 'not available' in these two settings upon creating the listing. 


In [76]:
#replace na values in each column by its key in the following dictionary
columns_to_na_replace= {'laundry_options' : 'no laundry on site',
 'parking_options' : 'no parking'}

import preprocessing as prepro

replace = prepro.ReplaceNA(columns_to_replacement= columns_to_na_replace) # Estimator for pipeline
new_data = replace.transform(data) #  Returns a copy of the database with the desired na replacements

#Check to see that we replaced each na value by the intended string (no new values made):

columns_replaced = ['laundry_options','parking_options']

for col in columns_replaced:
    new_vals = new_data[col].unique()
    old_vals = data[col].unique()
    for val in new_vals:
        if val not in old_vals:
            raise ValueError(f'{val} is a new value')
        else:
            pass
    
# The above for loop outputs nothing meaning that no new values were added, as desired

# Next we check that there are no na values

size_idx = new_data.index.size
for col in new_data.columns:
    na_col = new_data[col].isna()
    na_val = na_col.values 
    num_na = na_col.sum()
    
    print(f'{col} contains {num_na} na values. Approximately {num_na/size_idx} values in {col} are na.')

# The output of the above loop shows that there are no na values in our data set 
# Since new_data is precisely what we want, we rename it to data.

data = new_data

price contains 0 na values. Approximately 0.0 values in price are na.
sqfeet contains 0 na values. Approximately 0.0 values in sqfeet are na.
beds contains 0 na values. Approximately 0.0 values in beds are na.
baths contains 0 na values. Approximately 0.0 values in baths are na.
type contains 0 na values. Approximately 0.0 values in type are na.
cats_allowed contains 0 na values. Approximately 0.0 values in cats_allowed are na.
dogs_allowed contains 0 na values. Approximately 0.0 values in dogs_allowed are na.
smoking_allowed contains 0 na values. Approximately 0.0 values in smoking_allowed are na.
laundry_options contains 0 na values. Approximately 0.0 values in laundry_options are na.
parking_options contains 0 na values. Approximately 0.0 values in parking_options are na.
state contains 0 na values. Approximately 0.0 values in state are na.


As indicated the above cell, the DataFrame `data` now has each old na value in ``'laundry_options'`` and ``'parking_options'`` replaced by ``no laundry on site`` and ``no parking``, respectively. Our next step is to visualize the data 

 Next, we do feature engineering. 

1. We combine the features ``'cats_allowed'``, ``'dogs_allowed'``, ``'smoking_allowed'`` features by adding their values. Our justification here is that each of these features indicate a lifestyle choice.

2. We will combine the ``'beds'`` and ``'baths'`` features by adding their values and divide the sum by the value of ``'sqfeet'``. Our justification here is that the number of rooms (beds + baths for our purposes) divided by the square footage is an indicator of how the house is laid out. In particular a lower value in this new combined feature indicates that the house has more 'living room' than a higher value, which might be a useful insight. Moreover, it might be a good idea to scale the ``'beds'`` and ``'baths'`` column to higher values so that the resulting new columns values are not small numbers and can be more easily compared.

3. We will hotkey embedd``'type'``, ``'laundry_options'``, ``'parking_options'``, ``'state'``

The following code block will enact these changes

#